In [170]:
import pandas as pd

cars = pd.read_csv('scout_ErsteDaten.csv', error_bad_lines=False, delimiter=';')
brands = pd.read_csv('Marken.csv', header= None, usecols=[0,1,3, 4] , error_bad_lines=False, delimiter=';', encoding='ISO-8859-1')

In [171]:
cars["month"] = cars["data"].str.split('.').str[0]
cars["year"] = cars["data"].str.split('.').str[1]
cars.head()

,data,km,model,price,month,year
0,11.202,4'500 km,HONDA CR-V 2.0i MMD Hybrid Elegance 4WD Automatic,CHF 36'900.–,11,202
1,12.2013,153'200 km,MERCEDES-BENZ E 250 7G-Tronic,CHF 19'800.–,12,2013
2,3.202,6'900 km,BMW X3 M40d Individual Aut. 4x4 (CH),CHF 78'900.–,3,202
3,Neu,12 km,SUZUKI Swace 1.8 Hybrid Compact+,CHF 31'300.–,Neu,NaN
4,Neu,12 km,SUZUKI Ignis 1.2i Piz Sulai Top Hybrid 4x4,CHF 22'400.–,Neu,NaN


In [172]:
cars["year"].unique()
cars["year"][cars["year"]=='202']='2020'
cars["year"][cars["year"].isna()]='2021'
cars["month"][cars["month"]=="Neu"]='2'

In [79]:
type(cars[cars["year"].apply(len)==3]["year"])

pandas.core.series.Series

In [173]:
cars["km2"]=cars["km"].str.rstrip(' km')
cars['km2'] = [float(val.replace("'","").strip()) for val in cars['km2']]


In [174]:
cars["price2"]=cars["price"].str.lstrip('CHF ')
cars["price2"]=cars["price2"].str.rstrip('.–')
cars.head()

,data,km,model,price,month,year,km2,price2
0,11.202,4'500 km,HONDA CR-V 2.0i MMD Hybrid Elegance 4WD Automatic,CHF 36'900.–,11,2020,4500.0,36'900
1,12.2013,153'200 km,MERCEDES-BENZ E 250 7G-Tronic,CHF 19'800.–,12,2013,153200.0,19'800
2,3.202,6'900 km,BMW X3 M40d Individual Aut. 4x4 (CH),CHF 78'900.–,3,2020,6900.0,78'900
3,Neu,12 km,SUZUKI Swace 1.8 Hybrid Compact+,CHF 31'300.–,2,2021,12.0,31'300
4,Neu,12 km,SUZUKI Ignis 1.2i Piz Sulai Top Hybrid 4x4,CHF 22'400.–,2,2021,12.0,22'400


In [206]:
cars["model"]=cars["model"].str.lower()

In [1]:
brands = brands.set_axis(['brand', 'model', 'start_year', 'abc'], axis=1, inplace=False)
brands['brand'] = brands['brand'].str.lower() 
brands.head()

car_brands=brands["brand"].unique()

NameError: name 'brands' is not defined

In [218]:
for i in car_brands:
    print(i)
    if i in cars["model"]:
        print(1)
    else:
        print(0)


acura
0
alfa romeo
0
aston martin
0
audi
0
bentley
0
bmw
0
brilliance
0
bugatti
0
buick
0
byd
0
cadillac
0
changan
0
chery
0
chevrolet
0
chrysler
0
citroen
0
daewoo
0
daihatsu
0
datsun
0
dodge
0
dongfeng
0
faw
0
ferrari
0
fiat
0
fisker
0
ford
0
gaz
0
geely
0
genesis
0
gmc
0
great wall
0
haval
0
holden
0
honda
0
hummer
0
hyundai
0
infiniti
0
isuzu
0
iveco
0
jaguar
0
jeep
0
kia
0
lamborghini
0
lancia
0
land rover
0
lexus
0
lifan
0
lincoln
0
lotus
0
marussia
0
maserati
0
maybach
0
mazda
0
mclaren
0
mercedes
0
mercury
0
mg
0
mini
0
mitsubishi
0
nissan
0
noble
0
opel
0
peugeot
0
plymouth
0
pontiac
0
porsche
0
ravon
0
renault
0
rolls-royce
0
rover
0
saab
0
saturn
0
scion
0
seat
0
skoda
0
smart
0
ssang yong
0
subaru
0
suzuki
0
tesla
0
toyota
0
uaz
0
vaz
0
volkswagen
0
volvo
0


In [229]:
from re import search

fullstring = ["S tack Ab use", "absd"]
substring = ["tack", "abc"]

for i, j in substring:
    if search(i, j):
        print("substring")
    else:
        print("Not found!")

ValueError: too many values to unpack (expected 2)